# Projeto 2 de Ciência de dados:
 
 
 #### Introdução:
  Este projeto objetifica a tentativa de criação de um modelo de predição cuja variável de interesse é o Rating que um aplicativo da PlayStore recebe, de acordo com algumas caraceterísticas intrínsecas. O modelo fará isso utilizando um Dataset importado do Kaggle e um processe de regressão.
  Este projeto objetifica a criação de um modelo capaz de prever o Rating de aplicativos da Google Playstore, baseando-se em outras características, como quantidade de reviews, categoria e preço. O modelo utilizará um Dataset importado do Kaggle e será realizado um processo de regressão.
 
 #### Link do dataset:
  * https://www.kaggle.com/lava18/google-play-store-apps
 
#### Integrantes:
*  Jean Silas
*  Isabelle Moschini
*  Matheus Barros
*  Rafael Monteiro

#### Váriavel Target:
   * Ratings (0 a 5)
   
   
#### Features:
   * Category
   * Reviews
   * Installs
   * Price
   * Content Rating

# 1- Importando as bibliotecas e lendo a base de dados (dataset) :

O primeiro passo para começarmos a analisar o dataset é observar suas categorias e os tipos de elementos presentes em cada uma.

In [ ]:
#Importando as Bibliotecas:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import scipy.stats as stats 
import sklearn 
from math import *

In [ ]:
#Abrindo o Dataset
dataset = pd.read_csv('googleplaystore.csv')

#Verificando suas dimensões
print('Este dataset possui {} linhas e {} colunas'.format(dataset.shape[0],dataset.shape[1]))

In [ ]:
#Sondando os dados
dataset.head()

In [ ]:
#Sondando os tipos de dados
dataset.dtypes

  _Como o modelo utilizará métodos de regressão, precisaremos fazer um tratamento dos dados e priorizar pela transformação do máximo de elementos possível em numéricos._

# 2-Pré-processamento de dados:

  Após visualizar superficialmente os dados, vamos partir para a tratamento deles: vamos remover as colunas que não são interessantes a priori para a nosso modelo, remover espaços vazios e outros processos de limpeza.
  
  Nós não consideramos as colunas App, Size, Genres, Last Updated, Current Ver, Android Ver e Type interessantes para o modelo, uma vez que:
  * App: irrelevante e cada aplicativo possuía um valor diferente;
  * Size: como mostrado abaixo, a maioria dos dados tem valor "Varies with device", então essa categoria se torna irrelevante, pois grande parte dos valores, que deveriam ser numéricos, são desconhecidos;
  * Genres: repetitivo com Category, varia em um espectro maior e cada aplicativo pode possuir mais de uma;
  * Last Updated, Current Ver e Android Ver: seriam dados que precisariam de um tratamento e provavelmente não teriam muita influência sobre o modelo;
  * Type: seria repetitivo com o Price, que abrange diversos valores por não ser categórico. De qualquer forma, é interessante notar que a grande maioria dos aplicativos é gratuito.

In [ ]:
dataset.Size.value_counts()

In [ ]:
dataset.Type.value_counts()

In [ ]:
data0 = dataset.drop(columns=['App', 'Size', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver','Type'])
data0.head()

_É importantíssimo remover as linhas que possuem valores vazios para que não ocorra nenhum problema no momento de passar os dados pelo modelo de predição._

In [ ]:
#Estamos eliminando as linhas que possuem valores vazios
data0 = data0.dropna()
data0.head()

In [ ]:
data0.info()

_Uma vez retirado os valores nulos, vamos avaliar as frequências de cada variável, para assim entender como está o dataset e precisar os próximos passos de limpeza. Mas antes vamos tirar os cifões do coluna Price para e os "+" dos installs e transformar as variáveis Reviews, Installs e Price em númericas para poder aplicar as funções do Pandas_

In [ ]:
# Funções para tratar as colunas numéricas:

#Definindo função que limpa
def Price_clean(valor):
    valor_str = str(valor)
    return float(valor_str.replace('$', ''))

#Definindo função que limpa e transforma em valor numérico
def Installs_clean(valor):
    valor_str = str(valor)
    return int(valor_str.replace('+', '').replace(',',''))

In [ ]:
# Aplicando as funções definidas acima acima:
data0.Price = data0.Price.astype(str).map(Price_clean)
data0.Installs = data0.Installs.astype(str).map(Installs_clean)

In [ ]:
# Transformando todos os valores numéricos em float
data0.Reviews = data0.Reviews.astype('float64')
data0.Installs = data0.Installs.astype('float64')
data0.Price = data0.Price.astype('float64')
data0.dtypes

In [ ]:
data0.Category.value_counts()

In [ ]:
data0['Content Rating'].value_counts()

In [ ]:
data0.Rating.describe()

In [ ]:
data0.Reviews.describe()

In [ ]:
data0.Price.describe()

In [ ]:
data0.Installs.describe()

## 2.1- Decisões sobre o dataset:
    
   Após entendermos mais os dados decidimos tomas as seguintes decisões:
   
   * Retirar todas as linhas que possuem "Adults only 18+" como característica da feature Content Rating, uma vez que só possuem 3 linhas em que ele aparece, sendo assim, um outlier para o nosso modelo;
   
   * Retirar todas as linhas que possuem "Unrated" como característica da feature Content Rating, uma vez que só possui uma linha em que ele aparece, sendo assim, um outlier para o nosso modelo.

In [ ]:
data0 = data0.loc[data0['Content Rating'] != 'Adults only 18+', :]
data0 = data0.loc[data0['Content Rating'] != 'Unrated', :]
data0['Content Rating'].value_counts()

## 2.2- Dados tratados para fazer uma análise mais aprofundada:

   A partir desse ponto, já possuimos dados mais parecidos com o que usaremos no nosso modelo. Ess etapa inicial, consistiu em descartar dados que não são importantes para o modelo e para a análise exploratório e fazer a correção de dados necessária para a maior robustez do modelo e análise exploratória.
   
   * Váriavél Target:Rating
   * Features: Category, Reviews, Installs, Price e Content Rating.
   
   * Tipagem das features:
   
       * quantititvas: Reviews, Installs e Price.
       
       * qualitativas:  Category e Content Rating.

In [ ]:
data0.head()

In [ ]:
data0.dtypes

# 3- Analise exploratória do dataset:
   _Como o nosso dataset possui tanto variáveis qualitativas quanto quantitivas então separaremos as análises que cabem a cada tipode variável._

## 3.1 - Análise das variáveis quantitativas:

In [ ]:
#Entendendo as medidas das variáveis numéricas.
data0.describe()

In [ ]:
#Sondado a distribuição de Rating
plt.hist(data0.Rating,bins=42,edgecolor='white')
plt.title("Histograma dos ratings")
plt.xlabel("Rating")
plt.ylabel("Frequência")
plt.show()

In [ ]:
# O modelo aparenta ser uma distribuição normal, para isso verificaremos:
stats.probplot(data0.Rating,dist="norm",plot=plt)
plt.show()

_Com esse gráfico constatamos que a distribuição não é normal, e portanto nossa suposição estava errada_

In [ ]:
data0.Rating.describe()

_Aqui, podemos perceber que o valor de Rating mais frequente é de 4.4, seguido de suas vizinhanças. Quanto mais distante desse valor, menor é a frequência._

In [ ]:
#Sondado a distribuição de Reviews
plt.figure(figsize=(20, 10))

plt.subplot(221)
plt.hist(data0.Reviews,bins=42,edgecolor='white')
plt.title("Histograma dos reviews")
plt.xlabel("Reviews")
plt.ylabel("Frequência")

plt.subplot(223)
plt.hist(data0.Reviews,bins=42,edgecolor='white', range=[0, 1e3])
plt.title("Histograma dos reviews")
plt.xlabel("Reviews")
plt.ylabel("Frequência")
plt.show()

_Observando o primeiro gráfico, percebemos que Review está concentrado na vizinhança de um valor. Porém, reduzindo o intervalo analisado, notamos que os valores seguem alguma distribuição. Para verificar a situação, vamos analisar frequencia numericamente_

In [ ]:
#Verificando o valor 
data0.Reviews.describe()

_Os reviews não estão concentrados em um valor, mas sim em um intervalo_

In [ ]:
#Sondado a distribuição de Installs
plt.figure(figsize=(20, 10))

plt.subplot(221)
plt.hist(data0.Installs,bins=42,edgecolor='white', range=[0, 1e9])
plt.title("Histograma dos installs")
plt.xlabel("Installs")
plt.ylabel("Frequência")

plt.subplot(223)
plt.hist(data0.Installs,bins=42,edgecolor='white', range=[0, 1e6])
plt.title("Histograma dos installs")
plt.xlabel("Installs")
plt.ylabel("Frequência")
plt.show()

_Analisando o primeiro histograma, verificamos que a variável install se concentra na vizinhança de um valor. Porém, considerando um intervalo menor dos valores (de 0 a 1 000 000), percebemos que os dados estão distribuídos de forma inconclusiva. Vamos tentar entender isso, verificando os valores numericamente_

In [ ]:
#Sondando a situação
data0.Installs.describe()

_A quantidade de installs mais frequente dos aplicativos é da orderm de 10^3._

In [ ]:
#Sondado a distribuição de Price
plt.figure(figsize=(20, 10))

plt.subplot(221)
plt.hist(data0.Price,bins=62,edgecolor='white')
plt.title("Histograma dos prices")
plt.xlabel("Price")
plt.ylabel("Frequência")

plt.subplot(223)
plt.hist(data0.Price,bins=62,edgecolor='white', range=[0, 10])
plt.title("Histograma dos prices")
plt.xlabel("Price")
plt.ylabel("Frequência")

plt.show()

_Assim como o Install, o Price se concentra na vizinhança de um valor, que neste caso seria o zero. Com isso, podemos supor que a grande maioria dos apps são gratuitos. Para verificar tal suposição, verificamos a frequência dessa variável numricamente_

In [ ]:
#Verificando a suposiçaõ anterior:
data0.Price.describe()

In [ ]:
porcentagem = len(data0.loc[data0.Price==0,:])/len(data0)
print("Porcentagem de apps gratuitos é de : {}%".format(porcentagem*100))

_Com isso, confirmarmos a hipótese que grande parte dos aplicativos são gratuitos, já que 93% deles são Free_

In [ ]:
#Juntando os gráficos
plt.figure(figsize=(10, 10))

plt.subplot(221)
plt.hist(data0.Rating,bins=42,edgecolor='white',color='blue')
plt.title("Histograma dos ratings")
plt.xlabel("Rating")
plt.ylabel("Frequência")

plt.subplot(222)
plt.hist(data0.Price,bins=42,edgecolor='white',color='brown')
plt.title("Histograma dos reviews")
plt.xlabel("Reviews")
plt.ylabel("Frequência")

plt.subplot(223)
plt.hist(data0.Price,bins=42,edgecolor='white', color='red')
plt.title("Histograma dos prices")
plt.xlabel("Rating")
plt.ylabel("Frequência")

plt.subplot(224)
plt.hist(data0.Installs,bins=42,edgecolor='white', color='black')
plt.title("Histograma dos installs")
plt.xlabel("Rating")
plt.ylabel("Frequência")

plt.show()

## _Para entendermos ainda melhor essas 4 variáveis quantitavas, vamos plotar mais alguns gráficos:_
   _Assim podemos entender, por exemplo, a questão dos outliers._

In [ ]:
data0.boxplot(column='Rating');

In [ ]:
data0.boxplot(column='Reviews');

In [ ]:
data0.boxplot(column='Price');

In [ ]:
data0.boxplot(column='Installs');

Todos os gráficos apresentam uma grande quantidade de outliers. No caso do Price, em especial, os valores se aglomeram próximos a zero por terem muitos aplicativos gratuitos.

_E para terminar a análise das variáveis quantitativas, vamos verificar a correlação entre elas:_

In [ ]:
#Verificando entre as variáveis quantitativas
correl = pd.DataFrame.corr(data0)
correl

In [ ]:
#Verificando entre as variáveis quantitativas gráficamente
sns.heatmap(correl);

A fim de melhor observar os dados, obtendo um gráfico mais disperso e de fácil análise e comparação, foi aplicado um logaritmo de base 10 nos dados quantitativos que não o Rating, que é o valor a ser previsto.

In [ ]:
data0_log = data0.copy()
data0_log['log_price'] = np.log(data0_log['Price'] + 1)
data0_log['log_installs'] = np.log(data0_log['Installs'])
data0_log['log_reviews'] = np.log(data0_log['Reviews'])
data0_log = data0_log.drop(columns=['Price', 'Installs', 'Reviews'])

In [ ]:
# Verificando Graficamente:
sns.pairplot(data0_log);

Ao observar os gráficos de Price, percebemos que, em geral, os dados se agrupam em três partes diferentes: os aplicativos gratuitos ou muito baratos, os aplicativos muito caros e os aplicativos de preço mediano. Analisando-os, notamos que há uma pequena relação entre os Ratings e os preços, afinal, há uma tendência a aplicativos pagos terem avaliações acima da 2.5, porém não há uma relação direta entre esses valores. Os aplicativos gratuitos se distribuem por todos os Ratings, aparentemente sem um padrão.

Em relação aos Installs, este parece ter uma relação linear com os Reviews, o que faz sentido, pois quanto mais downloads um aplicativo possui, provavelmente mais avaliações terá. Ele também parece se relacionar com os Ratings, pois aplicativos com muitos downloads tendem a ter Ratings maiores, mais próximos de 5.0. 

Um comportamento muito semelhante ocorre com o gráfico dos Reviews em relação ao Rating, porém ainda mais acentuado, pois os aplicativos com muitas Reviews costumam ter um Rating muito próximo do valor máximo.


Porém, devido aos diferentes grupos identificados nos gráficos de Price, optamos por utilizar apenas os aplicativos gratuitos, pois, como visto anteriormente, englobam a maioria dos apps, representando cerca de 93%.

In [ ]:
data0 = data0_log.loc[data0_log.log_price == 0,:]
sns.pairplot(data0);

Após verificarmos que os aplicativos pagos foram retirados, deixamos de analisar a coluna que envolve os preços no modelo, pois ela não interferiria mais.

In [ ]:
data0 = data0.drop(columns=['log_price'])
sns.pairplot(data0);

## 3.2- Conclusões a respeito da análise exploratória das variáveis quantitivas:

   O que se percebe é que a nossa variável target não é muito bem explicada pelas variáveis quantitaivas, o que pode ser ruim para o modelo mais a frente, já que a baixa correlação entre as features e o target pode resultar num modelo que não é eficiente e que não preve o que ele foi criado para prever. Nos resta entender como as variáveis qualitativas se correlacionam com a variável Target.

## 3.3-Análise exploratória: entendendo os dados qualitativos
   _Agora analisaremos as váriaveis qualitativas: Category e Content Rating_ 

In [ ]:
#Verificando a frequência de cada classe de Category:
data0.Category.value_counts()

In [ ]:
#Verificando a frequência de cada classe de Content Rating:
data0['Content Rating'].value_counts()

In [ ]:
#Verificando a frequência de cada classe de Category graficamente
plt.figure(figsize=(40, 40))
sns.countplot(data0.Category);

In [ ]:
#Verificando a frequência de cada classe de Content Rating graficamente
sns.countplot(data0['Content Rating']);

In [ ]:
labels =['Everyone','Teen','Mature 17+','Everyone 10+']
colors = ['red','orange','green','blue']
i = 0
for label in labels:
    data = dataset.loc[dataset['Content Rating']==label,:]
    data = data['Rating']
    plt.hist(data,bins=42, label=label,color=colors[i], edgecolor='white')
    plt.legend()
    plt.xlabel('\nRating')
    plt.ylabel('Contagem de Rating\n')
    plt.title('\nRating por {}\n'.format(label))
    plt.show()
    i+=1

In [ ]:
labels =['FAMILY','GAME','TOOLS','PRODUCTIVITY','MEDICAL','COMMUNICATION','FINANCE','SPORTS','PHOTOGRAPHY','LIFESTYLE','PERSONALIZATION','BUSINESS','HEALTH_AND_FITNESS','SOCIAL','SHOPPING','NEWS_AND_MAGAZINES','TRAVEL_AND_LOCAL','DATING','BOOKS_AND_REFERENCE','VIDEO_PLAYERS','EDUCATION','ENTERTAINMENT','MAPS_AND_NAVIGATION','FOOD_AND_DRINK','HOUSE_AND_HOME','WEATHER','AUTO_AND_DEMO','LIBRARIES_AND_DEMO','ART_AND_DESIGN','COMICS','PARENTING','EVENTS','BEAUTY']
colors = ['red','orange','green','blue']
i = 0
for label in labels:
    data = dataset.loc[dataset['Category']==label,:]
    data = data['Rating']
    plt.hist(data,bins=42, label=label,color=colors[i], edgecolor='white')
    plt.xlabel('\nRating')
    plt.ylabel('Quantidade de aplicativos\n')
    plt.title('\nRating por {}\n'.format(label.lower()))
    plt.legend()
    plt.show()
    i+=1
    i = i%4

Após a análise dos dados qualitativos, chegou o momento de tranformá-los em dados numéricos, por meio dos dummies. Afinal, por estarmos utilizando um modelo de regressão, é necessário estabelecer como input apenas dados quantitavos.

O processo de transformação de variáveis qualitativas para variáveis dummy se dá por meio da alocação de cada classe da feature em questão e valores de 0 e 1. Cada classe dessa feature se torna uma coluna nova, em que a classe correspondente à linha em questão assuma o valor 1, e todas as outras colunas a essa linha tenham valor 0.

Um adendo: quando a feature possui k classes, o valor de colunas geradas ideal é k - 1.

In [ ]:
#Vamos transformar as variáveis catégoricas em dummies:
df_d = pd.get_dummies(data0, columns=['Category','Content Rating','log_installs'], drop_first=True)
print('Este dataset possui {} linhas e {} colunas'.format(df_d.shape[0],df_d.shape[1]))

In [ ]:
df_d.head()

# Regressão Linear

O primeiro processo de regressão pelo qual nosso dataset irá passar é o de regressão linear (linear regression).

Primeiramente, utilizamos o método de regressão linear. Esse método consiste em associar uma variável dependente Y em função de uma ou mais variáveis independentes X. Assim, pode-se estabelecer uma função de primeiro grau a fim de resumir essa relação e que possa ser utilizada para predizer os valores de Y.

Para calcular os coeficientes dessa reta, é necessário utilizar o Método dos Mínimos Quadrados (MMQ). Tal método busca o melhor valor que os coeficientes possam atingir, de maneira que a diferença entre o valor predito pela função e o valor real, seja a menor possível.

Dessa forma, ao observarmos que existia uma relação em um diagrama de dispersão entre o Rating e os Installs e Reviews, optamos por utilizar a regressão linear para fazer previsões a partir dos dados.

A Variável dependente (Y) é a variável que queremos prever, que corresponde ao Rating dos aplicativos no caso do nosso projeto. Já as variáveis independentes (X), são aquelas que podem influenciar na variável que será prevista. Nesse caso, serão todas as informações do nosso DataSet, ou seja, Installs, Reviews, Category, entre outros.

In [ ]:
import statsmodels.api as sm

In [ ]:
#Criando função do modelo por regressão linear
def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

In [ ]:
train_ = df_d.loc[:7800,:]
test = df_d.loc[7801:,:]

In [ ]:
Y = train_['Rating']
X = train_.drop(columns=['Rating'])
Ytest = test['Rating']
Xtest = test.drop(columns=['Rating'])

In [ ]:
results = regress(Y,X)

pre = results.predict(test)

In [ ]:
coefb = results.params

In [ ]:
#Apresentando os resultados
results.summary()

Para tirar alguma conclusão sobre a eficácea desse modelo, optamos por observar o valor de 'R-Squared', que representa um valor entre 0 e 1, sendo melhor quanto mais próximo do valor máximo. No caso do nosso modelo, esse valor é de 0.191, o que é baixo. 

Portanto, podemos concluir que a regressão linear não é o melhor método para tentar prever o Rating dos aplicativos baseado no dataset utilizado. Outra hipótese seria que o Rating dos aplicativos, considerando os demais dados apresentados, é imprevisível.

Assim, decidimos utilizar outro método para regressão, o Random Forest.

# Random Forest

Random forest é uma técnica usada em machine learning, pode ser aplicada tanto em regrassão quanto classificação, no nosso caso usamos regressão por se tratar de um modelo quantitativo.

A aplicação da técnica é similar à um processo de decisão em conjunto em que o dataset é percorrido por diferentes caminhos resultando em um valor por àrvore, posteriormente sendo combinadas múltiplas árvores de decisão e tirado a média entre elas a fim de ter um resultado fiel ao dataset analisado.

É recomendado que os valores analisados não sejam inviesados, caso os tenha é muito provável que a análise retornara um valor promissor no entanto por ser ilusório pois as árvores acabaram tomando o mesmo caminho diversas vezes devido à maior concentração em um determinado valor.

Tomamos certas precauções para que os valores do dataset sejam realocados nas árvores de maneira eficiente e que o resultado seja mais próximo da realidade, seguindo essa linha definimos a quantidade de estimadores à aproximadamente 100 que não é um valor exacerbado, utilizamos logaritmo para os valores de review e installs, e limitamos a análise para apenas os aplicativos gratuitos, para melhor compreensão e análise.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression

In [ ]:
#Plotando o modelo por Random Forest
model = RandomForestRegressor(n_estimators=100,n_jobs=-1)

model.fit(X, Y)

In [ ]:
#Criando a variável que prevê o valor do Rating, baseando-se nas demais características do teste
Ypred = model.predict(Xtest)

Para verificar a performance de nosso modelo com Random Forest, usamos a função .score(). Primeiro, comparamos o valor previsto pelo modelo com o esperado pelo conjunto de treino, usando X e Y de treino.

In [ ]:
model.score(X, Y)

Então, usamos o score() novamente, dessa vez para comparar o valor previsto pelo modelo com o esperado, no conjunto de testes.

In [ ]:
model.score(Xtest, Ytest)

Na função .score(), quanto mais próximo de 1 o resultado, mais eficiente é o modelo dado o dataset utilizado. No primeiro caso, o valor está relativamente próximo de 1.0, porém, no segundo, está bem distante (inclusive, é negativo).

Essa enorme diferença revela, portanto, que o modelo tem um problema de Overfitting. Ou seja, ele é capaz de prever valores bem próximos aos reais utilizando dados do treino, porém é incapaz de fazer boas previsões com valores nunca antes vistos, que é o caso do teste.

A fim de verificar a eficiência desse modelo na predição, utilizamos outros métodos, como o R Squares, que também foi utilizado na Regressão linear:

In [ ]:
rmse = (sklearn.metrics.mean_squared_error(Ytest, Ypred))**(1/2)
rmse

Essa verificação aponta que o modelo não está tão ruim quanto a verificação anterior. Afinal, o valor apontado é de mais de 0.6 sendo que o maior possível  e o melhor de 1.0.

Porém, antes de chegar a qualquer conclusão, é importante verificar essa eficiência de outras formas, como utilizando os resíduos, que demonstram a diferença entre o valor do Rating previsto pelo modelo e o real, apontado nos dados de teste.

In [ ]:
residuos = Ytest - Ypred

In [ ]:
plt.hist(residuos, bins=50);
plt.title('Histograma dos resíduos do Rating\n')
plt.ylabel('Contagem dos resíduos\n')
plt.xlabel('\nValor dos resíduos')
plt.show()

In [ ]:
residuos.std(ddof=1)

Observando o histograma plotado, podemos perceber que há diferenças consideráveis entre os valores (chegando até mais de 2, em módulo, em alguns casos). Afinal, o valor de Rating varia entre 0 e 5 apenas, o que significa que há um desvio padrão alto, de cerca de 12% em relação aos valores possíveis.

Então, plotamos histogramas e um gráfico de correlação a fim de comparar o Rating previsto pelo modelo com seus valores presentes nos dados de teste.

In [ ]:
plt.hist(Ytest, bins=50);

In [ ]:
plt.hist(Ypred, bins=50);

In [ ]:
plt.scatter(Ytest, Ypred)

In [ ]:
#Comparação numérica entre os valores de Rating do teste e previsto pelo modelo.
#Quanto mais próximo de 1, melhor

from sklearn.metrics import r2_score
r2_score(Ytest, Ypred)

# Conclusão

De início, nossa intenção era, com base nos dados do dataset, estipular o Rating dos aplicativos do Google Playstore. Ao decorrer do projeto, nos deparamos com certas variáveis que não possuíam uma correlação clara e interessante para prever os valores desejados.

Durante o processo, realizamos um pré-processamento dos dados do dataset, fizemos uma análise exploratória deles, comparamos as características escolhidas por meio de gráficos, transformamos variáveis qualitativas em dummies e utilizamos o log para deixar as variáveis em uma escala mais conveniente para fazermos as análises.

Por fim, realizamos um modelo por Regressão Linear, que não se mostrou eficiente em prever o Rating. Então, optamos por utilizar um modelo de regressão com Random Forest, que também foi incapaz de realizar boas previsões.

Com isso, nós concluímos que, baseando-se apenas nas características presentes no dataset utilizado e escolhidas para o nosso modelo, o valor do Rating é imprevisível. Afinal, não foi possível relacioná-lo de maneira direta a nenhuma variável analisada nesse projeto, portanto ele provavelmente se relaciona a diversas outras variáveis externas não presentes nesse dataset.

## Referências:
* https://escholarship.org/uc/item/35x3v9t4
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
* https://medium.com/data-hackers/implementando-regress%C3%A3o-linear-simples-em-python-91df53b920a8
* http://www.uel.br/pessoal/lscunha/pages/arquivos/uel/Economia%20Noturno%202018/Aula%207%20-%20Regress%C3%A3o%20Linear%20M%C3%BAltipla.pdf
* https://stattrek.com/multiple-regression/dummy-variables.aspx
* https://medium.com/@alegeorgelustosa/an%C3%A1lise-explorat%C3%B3ria-e-preditiva-do-dataset-titanic-em-python-fbd5e5bb7328
* https://www.youtube.com/watch?v=0s_1IsROgDc
* https://medium.com/swlh/random-forest-and-its-implementation-71824ced454f
* https://medium.com/towards-artificial-intelligence/exploratory-data-analysis-in-python-ebdf643a33f6

## Participação de cada membro do grupo


No começo do trabalho a equipe toda se manejou para procurar a temática do que seria analisado, tivemos em mente alguns temas e acabamos escolhendo em equipe "aplicativos da google play" a fim de descobrir o rating baseado no que o dataset tinha disponível.



Acabamos tendendo à regressão por se tratar de um valor analisado quantitativo.



Primeiramente a Isabelle compartilhou a tela e em conjunto com os outros integrantes extraiu o dataset e começou a fazer o cleanup dos dados, dropamos coisas desnecessárias como tamanho do arquivo (pois variava de acordo com o dispositivo), versão, nome e gênero.



Paralelamente o Jean pegou os valores já limpos, fez a implementação dos gráficos de análise exploratória e começou a introdução de transformar as variáveis categóricas em Dummies.



Isabelle, Rafael e Matheus analisaram os passos feitos pelo Jean explicando verificando certas partes da análise exploratória e mudando alguns dos gráficos a fim de melhor analisá-los, como por exemplo ajustar alguns eixos.



Jean compartilhou a tela e em conjunto foi incializado a primeira implementação de regressão linear com os dados da análise exploratória.



Matheus compartilhou a tela e em conjunto foi analisado os resultados da regressão linear e verificação da eficácia do modelo, foi então tomado um consenso, junto com o professor Ayres, de se tentar por meio de um modelo random forest para ver se previsão iria melhorar.



Rafael compartilhou a tela junto com o grupo, foi mudadoi alguns valores a fim de melhorar a análise, melhor implementação da regressão linear e foi constatado que mesmo com as coisas tomadas não era possível definir Rating com base nos dados que nós analisamos.



Isabelle compartilhou a tela, foi feito o filtro de valores como installs e reviews usando logaritmo, indpendente dos casos foi impossibilitado definir o rating com base nas variáveis analisadas.



Rafael, Isabelle, Jean e Matheus olharam as rúbricas expostas no excel e separaram as coisas que faltava para atingir conceito A, cada um ficou responsável por uma parte:
* Rafael, explicação linear regression.
* Isabelle, manejamento do jupyter e explicação de partes que faltavam.
* Matheus, explicação do random forest.
* Jean, comparação das variáveis qualitativas com a target.



Por fim todos os membros discutiram e decidiram a conclusão do projeto.



Agradecimento ao Maciel e Ayres por nos ajudarem nas situações de dificuldade, ajudaram bastante na implementação dos modelos e fizeram a gente tomar consciência de certos pontos do nosso trabalho.